In [2]:
cd

/Users/karatechop


In [3]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022\ \(CSV\)

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022 (CSV)


In [4]:
df = pd.read_csv('working_main_expanded.csv')
print(df.columns,'\n',df.shape)

Index(['date', 'day_in_year', 'time', 'sunsettime', 'dusktime', 'sex', 'race',
       'type', 'duration', 'agency', 'location', 'city', 'lat', 'lon', 'wave',
       'mins_stop_time', 'mins_sunset', 'mins_dusk', 'mins_after_dusk',
       'dark'],
      dtype='object') 
 (2408246, 20)


In [6]:
itp = df[(df['type']!='Non-Moving') &
        ((df['mins_stop_time']>=990) & (df['mins_stop_time']<=1230)) & #filtering for ITP (4:30-8:30)
        ((df['mins_after_dusk']<=-30) | (df['mins_after_dusk']>=0))][['date', #removing sunset->dusk period
                                                                      'race',
                                                                      'location',
                                                                      'city',
                                                                      'mins_after_dusk',
                                                                      'dark']]

itp.shape

(340906, 6)

## VOD 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
from postal.parser import parse_address
from postal.expand import expand_address

### Statewide VOD Model

In [9]:
itp.columns

Index(['date', 'race', 'location', 'city', 'mins_after_dusk', 'dark'], dtype='object')

In [10]:
#https://www.geeksforgeeks.org/ml-one-hot-encoding-of-datasets-in-python/
vod = pd.get_dummies(itp, columns = ['race'], drop_first=True, prefix='')\
                    [['date',
                      'location',
                      'city',
                      'mins_after_dusk',
                      'dark',
                      '_black/african american']].rename(columns={'_black/african american':'black'})\
                                                 .reset_index(drop=True)
vod.head(2)
vod.shape

(340906, 6)

In [341]:
la = vod[vod['city'] == 'Los Angeles'].reset_index(drop=True)
la.head(2)
la['location'] = [i.lower().lstrip('').rstrip('') for i in la['location']]
locations_la = la['location'].unique()
len(locations_la)

57260

In [356]:
# la['location'].replace('saint','street',inplace=True)

la['location'] = [i.replace('s ','') for i in la['location'] if len(i.split()[0]) == 1]
la['location'] = [i.replace('e ','') for i in la['location'] if len(i.split()[0]) == 1]
la['location'] = [i.replace('n ','') for i in la['location'] if len(i.split()[0]) == 1]
la['location'] = [i.replace('w ','') for i in la['location'] if len(i.split()[0]) == 1]

la['location'] = la['location'].str.replace(' s ','')
la['location'] = la['location'].str.replace(' n ','')
la['location'] = la['location'].str.replace(' e ','')
la['location'] = la['location'].str.replace(' w ','')
# la['location'] = la['location'].str.replace('san ','')

# expand_start = time()
# la['expanded_location'] = [expand_address(i)[0] for i in la['location']]

# la['location'] = la['location'].str.replace('saint','')
# la['location'] = la['location'].str.replace(' st ','')
# la['location'] = la['location'].str.replace('san ','')
la['location'] = la['location'].str.replace('south ','')
la['location'] = la['location'].str.replace('north ','')
la['location'] = la['location'].str.replace('east ','')
la['location'] = la['location'].str.replace('west ','')




# print(f'debug,expand worked and took {round((time()-expand_start)/60,2)} minutes')

addresses = [i.split('&')[0].rstrip('').lstrip('') + ', los angeles' for i in la['location']]
# addresses = [i.split('&')[0].replace(' - los angeles','').rstrip('').lstrip('') for i in la['location']]

len(pd.unique(addresses))

SyntaxError: invalid syntax (3033127843.py, line 3)

In [378]:
from geopy import Point
from geopy import Nominatim
from postal.parser import parse_address
from postal.expand import expand_address
import googlemaps
from key import key #this is a file ignored in the REPO to privately host my google maps API key

key = key.replace('†','')
gmaps = googlemaps.Client(key=key)
# https://console.cloud.google.com/apis/credentials?project=la-api-key


 API queries_quota: 60 



In [397]:
str(gmaps.geocode('88th place',bounds=la_bounds)[0]['geometry']['location']['lat']) + ' , ' +\
str(gmaps.geocode('88th place',bounds=la_bounds)[0]['geometry']['location']['lng'])

'33.9565039 , -118.2625641'

In [360]:
la_viewbox = ( (33.525861509262725,-117.32095033303995), (34.26915279876192,-118.52290082214722) )   
la_bounds = gmaps.geocode('los angeles')[0]['geometry']['bounds']

def find_lat_lon(address):
    street = address.split('&')[0]
    
    try:
        geolocator = Nominatim(user_agent="kcharris@berkeley.edu")
        location = geolocator.geocode(street,viewbox=la_viewbox,bounded=True)
    
        location_info = location.raw
        coords = location_info['display_name'].split()[-3].replace(',','')
    
    except:
        
        try:
#             coords = int(gmaps.geocode(street,bounds=la_bounds)[0]['address_components'][-1]['long_name'])
            coords = str(gmaps.geocode('88th place',bounds=la_bounds)[0]['geometry']['location']['lat']) + ' , ' +\
                     str(gmaps.geocode('88th place',bounds=la_bounds)[0]['geometry']['location']['lng'])
        
        except:
            print(f'google maps failed for {street}')
            pass
    
    return coords

ApiError: REQUEST_DENIED (The provided API key is invalid. )

In [339]:
dog = zip_finder_replacer(addresses[:50])
dog
# la.to_csv('la.csv',index=False)

there are 36 starting unique values.
google maps failed for vinst , los angeles
google maps failed for abbot kinney , los angeles
google maps failed for vanuyblvd , los angeles
google maps failed for washingtoblvd , los angeles
google maps failed for washingtoblvd , los angeles
google maps failed for washingtoblvd , los angeles
google maps failed for washingtoblvd , los angeles
google maps failed for mlk blvd , los angeles
google maps failed for lincolblvd , los angeles
google maps failed for vernoav, los angeles
google maps failed for 42nd pl , los angeles
google maps failed for venicblvd , los angeles
processing took 1.08 minutes. 
There are now 36 unique values.


[2602,
 'vinst , los angeles',
 '56th st , los angeles',
 'abbot kinney , los angeles',
 'palm, los angeles',
 'pacific av, los angeles',
 'kewe, los angeles',
 'wilcox av, los angeles',
 'sunset , los angeles',
 'calvert st , los angeles',
 'vanuyblvd , los angeles',
 'washingtoblvd , los angeles',
 'washingtoblvd , los angeles',
 'washingtoblvd , los angeles',
 'washingtoblvd , los angeles',
 'sepulveda blvd , los angeles',
 'florenc, los angeles',
 'mlk blvd , los angeles',
 'brookav, los angeles',
 'lincolblvd , los angeles',
 'budlong av, los angeles',
 '7000 beviav- loangeles, los angeles',
 '7000 beviav- loangeles, los angeles',
 '4000 nicolet av- loangeles, los angeles',
 '59th pl , los angeles',
 '59th pl , los angeles',
 '59th pl , los angeles',
 '59th pl , los angeles',
 '59th pl , los angeles',
 '59th pl , los angeles',
 'pacific av, los angeles',
 '4800 slausoav- loangeles, los angeles',
 'vernoav, los angeles',
 '19th st , los angeles',
 'cabrillo av, los angeles',
 '88th

In [186]:
# zip_start = time()
# la_viewbox = ( (33.525861509262725,-117.32095033303995), (34.26915279876192,-118.52290082214722) )   
# street = 'james m wood boulevard & south new hampshire avenue los angeles'
# street = street.split('&')[0]

# geolocator = Nominatim(user_agent="kcharris@berkeley.edu")
# location = geolocator.geocode(street,viewbox=la_viewbox,bounded=True)

# test_zip = location.raw
# test_zip = test_zip['display_name'].split()[-3].replace(',','')
# print(test_zip,'time:',round(time()-zip_start,2),'seconds')

james m wood boulevard 
90017 time: 1.13 seconds
